# Convolution

Convolution is a mathematical way of combining two signals to form a third signal. It is the single most important technique in Digital Signal Processing. Using the strategy of impulse decomposition, systems are described by a signal called the impulse response. Convolution is important because it relates the three signals of interest: the input signal, the output signal, and the impulse response. 

This chapter presents 1D and 2D convolution. For 1D convolution two different viewpoints, called the **input side algorithm** and the **output side algorithm**. are shown, then a vectorized implementation is presented. For 2D convolution a vectorized form is presented applied to image processing.

## 1D Convolution
The mathematical form of the convolution is:

$$ y[i] = \sum_{j=0}^{M-1}{x[j]h[i-j]} $$

To develop the convolution we define the following:
    
* Input Signal $x[n]$ of size $N$ 
* Impulse Response $h[n]$ of size $M$
* Output Signal $y[n]$ of size $N + M -1$

There are two types of algorithms that can be performed:

1. Output Side Algorithm
2. Input Side Algorithm

### 1. Output Side Algorithm
Analyzes how each sample in the input signal affects many samples in the output signal. (We sum the contributions of each input to every output sample.)

![Input Side Algorithm](Images/input_side_algorithm.gif)

The algorithm calculates the convolution in the following way:


$$y[n] = \sum_{i=0}^{N-1}  \sum_{j=0}^{M-1}{x[i]h[j]}$$ 

where $n = i+j$, $M$ is the length of the impulse response and $N$ the input signal size and $y[n]$ has a size of $M+N-1$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pickle

In [ ]:
file = {'x':'Signals/InputSignal_f32_1kHz_15kHz.dat', 'h':'Signals/Impulse_response.dat'}

x = np.loadtxt(file['x'])
N,M = x.shape
x = x.reshape(N*M, 1)

h = np.loadtxt(file['h'])
N = h.shape[0]
h = h.reshape(N, 1)

In [ ]:
def convolve_output_algorithm(x, h):
    """ 
    Function that convolves an input signal x with an step response h using the output side algorithm.
  
    Parameters: 
    x (numpy array): Array of numbers representing the input signal to be convolved.
    h (numpy array): Array of numbers representing the unit step response of a filter.
  
    Returns: 
    numpy array: Returns convolved signal y[n]=h[n]*x[n].
  
    """
    M = h.shape[0]
    N = x.shape[0]

    output = np.zeros(M+N-1)
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return output.reshape(-1,1)    

In [ ]:
output = convolve_output_algorithm(x, h)

assert np.isclose(output, (np.convolve(x.reshape(-1), h.reshape(-1))).reshape(-1,1)).all()

In [ ]:
plt.rcParams['figure.figsize'] = (12,12)

plt.subplot(2,2,1)
plt.plot(x)
plt.title('Input Signal')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid('on')

plt.subplot(2,2,2)
plt.plot(h)
plt.title('Filter Impulse Response')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid('on')

plt.subplot(2,1,2)
plt.plot(output)
plt.title('Output Signal, Output Side Algorithm')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid('on')
plt.show()

### 2. Input Side Algorithm
We look at individual samples in the output signal and find the contributing points from the input. (We find who contributed to the output.)

The algorithm calculates the convolution in the following way:

[//]: $$y[i] = \sum_{i=0}^{M+N-1}  \sum_{j=0}^{M-1}{h[j]x[i-j]}$$ 
$$y[i] = \sum_{j=0}^{M-1}{h[j]x[i-j]}$$ 

if $$i-j>0 $$ and $$i-j<N-1$$

where $M$ is the length of the impulse response and $N$ the input signal size and $y[n]$ has a size of $M+N-1$.

In [ ]:
def convolve_input_algorithm(x, h):
    """ 
    Function that convolves an input signal x with an step response h using the input side algorithm.
  
    Parameters: 
    x (numpy array): Array of numbers representing the input signal to be convolved.
    h (numpy array): Array of numbers representing the unit step response of a filter.
  
    Returns: 
    numpy array: Returns convolved signal y[n]=h[n]*x[n].
  
    """
    M = h.shape[0]
    N = x.shape[0]

    output = np.zeros(M+N-1)
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return output.reshape(-1,1)    

In [ ]:
output_ = convolve_input_algorithm(x, h)

assert np.isclose(output_, (np.convolve(x.reshape(-1), h.reshape(-1))).reshape(-1,1)).all()

In [ ]:
plt.rcParams['figure.figsize'] = (12,12)

plt.subplot(2,2,1)
plt.plot(x)
plt.title('Input Signal')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid('on')

plt.subplot(2,2,2)
plt.plot(h)
plt.title('Filter Impulse Response')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid('on')

plt.subplot(2,1,2)
plt.plot(output_)
plt.title('Output Signal, Input Side Algorithm')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid('on')

plt.show()

### Comparison Between Speeds of Both Algorithms
`%timeit` is an ipython magic function, which can be used to time a particular piece of code (A single execution statement, or a single method).

In [ ]:
%timeit output = convolve_output_algorithm(x, h)

In [ ]:
%timeit output = convolve_input_algorithm(x, h)

### 3. A Faster 1D Convolution
A faster 1D convolution can be performed if inner loops can be transformed into matrix multiplications. This task can be accomplished by using *Toeplitz* matrices. A Toeplitz matrix or diagonal-constant matrix, named after Otto Toeplitz, is a matrix in which each descending diagonal from left to right is constant. For instance, the following matrix is a Toeplitz matrix: 

In [ ]:
from scipy.linalg import toeplitz
print(toeplitz(np.array([[1,2,3,4,5]])))

1D convolution can be obtained by using the lower triangular matrix of the Toeplitz matrix, $H$, and the vector $x$. For the matrix $H$ and vector $x$ to have right dimensions, zero padding must be used. The lower triangular matrix can be calculated using `np.tril()`.

In [ ]:
print(np.tril(toeplitz(np.array([[1,2,3,4,5]]))))

In [ ]:
def conv1d(x, h):
    """ 
    Function that convolves an input signal x with an step response h using a Toeplitz matrix implementation.
  
    Parameters: 
    x (numpy array): Array of numbers representing the input signal to be convolved.
    h (numpy array): Array of numbers representing the unit step response of a filter.
  
    Returns: 
    numpy array: Returns convolved signal y[n]=h[n]*x[n].
  
    """
    N = x.shape[0]
    M = h.shape[0]
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
output_fast = conv1d(x, h)

assert np.isclose(output_fast, (np.convolve(x.reshape(-1), h.reshape(-1))).reshape(-1,1)[:-h.shape[0]+1]).all()

In [ ]:
%timeit output_fast = conv1d(x, h)

In [ ]:
plt.rcParams['figure.figsize'] = (12,12)

plt.subplot(2,2,1)
plt.plot(x)
plt.title('Input Signal')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid('on')

plt.subplot(2,2,2)
plt.plot(h)
plt.title('Filter Impulse Response')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid('on')

plt.subplot(2,1,2)
plt.plot(output_fast)
plt.title('Output Signal, Fast Algorithm')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.grid('on')
plt.show()

## 2D Convolution on Images
If the convolution is performed between two signals spanning along two mutually perpendicular dimensions (i.e., if signals are two-dimensional in nature), then it will be referred to as 2D convolution. This concept can be extended to involve multi-dimensional signals due to which we can have multidimensional convolution.

For a 2D filter $h[m,n]$, or *kernel*, that has size $2M$ by $2N$ a 2D convolution is defined as follows:

$$y[i,j]=\sum_{m=-M}^{M+1}\sum_{n=-N}^{N+1}{h[m,n]x[i-m,j-n]}$$

In [ ]:
def conv2d(image, kernel):
    """ 
    Function that convolves an input image with a filter kernel.
  
    Parameters: 
    image (numpy matrix): Matrix representing a 2D image.
    kernel (numpy array): An m by n matrix to apply.
  
    Returns: 
    numpy matrix: Returns convolved image with filter kernel.
  
    """
    M, N = (np.array(kernel.shape)/2).astype('int')
    
    m , n = image.shape
    output = np.zeros((m,n))
    
    # YOUR CODE HERE
    raise NotImplementedError()
    return output

In [ ]:
from PIL import Image

# Load original image into image_original variable.
# YOUR CODE HERE
raise NotImplementedError()


# Convert image into gray scale and save it on image_gray variable. 
# To turn the image into gray you can search for the Image.convert() method.
# YOUR CODE HERE
raise NotImplementedError()


# Resize the gray image and store it in image_resize, you can search for the
# Image.resize() method.
scale_factor = 2 #scaling factor to use
# YOUR CODE HERE
raise NotImplementedError()

# Set image as an 2d-array x
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
Sx = np.array([[-1, 0, 1],[-2, 0, 2], [-1, 0, 1]])
Sy = np.array([[-1, -2, -1],[0, 0, 0], [1, 2, 1]])


Gx_2 = conv2d(x, Sx)
Gy_2 = conv2d(x, Sy)
image_output = np.sqrt(np.power(Gx_2,2) + np.power(Gy_2,2))


with open('image.pkl', 'rb') as file:
    image_pickle = pickle.load(file)
    
assert np.isclose(image_output, image_pickle).all()

In [ ]:
plt.subplot(1,3,1)
plt.imshow(image_original.resize((p,q)))
plt.title("Original")

plt.subplot(1,3,2)
plt.imshow(image_output, cmap='gray', vmin=0, vmax=255);
plt.title("Implemented")

plt.subplot(1,3,3)
plt.imshow(image_pickle, cmap='gray', vmin=0, vmax=255);
plt.title("Reference")

#### References:

* http://www.dspguide.com/ch6.htm
* https://www.allaboutcircuits.com/technical-articles/two-dimensional-convolution-in-image-processing/
* https://www.aishack.in/tutorials/image-convolution-examples/